<a href="https://colab.research.google.com/github/MahdiFaourr/MahdiFaourr/blob/main/makeup_nomakeup_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets

In [7]:
# Import necessary libraries and functions
import pandas as pd
import numpy as np
import opendatasets as od
import os
import shutil
from keras.metrics import Precision
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input, decode_predictions
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [ ]:
od.download("https://www.kaggle.com/datasets/petersunga/make-up-vs-no-make-up")

In [4]:
# Set some directories
classes = ['makeup', 'no_makeup']
train_directory = "/content/train"
test_directory = "/content/test"
root_directory = "/content/make-up-vs-no-make-up/data"

# Iterate over classes
for class_name in classes:
    class_directory = os.path.join(root_directory, class_name)
    images = os.listdir(class_directory) # List all images in the targeted path

    # Split the images into training and testing sets
    train_images, test_images = train_test_split(images, test_size=0.1)

    # Create destination directories
    train_class_dir = os.path.join(train_directory, class_name)
    os.makedirs(train_class_dir, exist_ok=True)

    test_class_dir = os.path.join(test_directory, class_name)
    os.makedirs(test_class_dir, exist_ok=True)

    # Copy training images
    for image in train_images:
        source_path = os.path.join(class_directory, image)
        destination_path = os.path.join(train_class_dir, image)
        shutil.copy(source_path, destination_path)

    # Copy testing images
    for image in test_images:
        source_path = os.path.join(class_directory, image)
        destination_path = os.path.join(test_class_dir, image)
        shutil.copy(source_path, destination_path)

In [8]:
# Set the ImageDataGenerator for data augmentation
datagen=ImageDataGenerator(rescale=1.0/255.0,
      rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2)

train_generator=datagen.flow_from_directory("/content/train",class_mode="binary",batch_size=32,target_size=(224,224))
test_generator=datagen.flow_from_directory("/content/test",class_mode="binary",batch_size=32,target_size=(224,224))

Found 1354 images belonging to 2 classes.
Found 152 images belonging to 2 classes.


In [18]:
precision_metric=Precision()

In [29]:
# Load MobileNetV1 base model without top (fully connected) layers
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a Sequential model
model = Sequential()

# Add the MobileNetV1 base model to the Sequential model
model.add(base_model)
# Set MobileNetV1 layers as non-trainable
for layer in base_model.layers:
    layer.trainable = False

# Add GlobalAveragePooling2D layer to reduce spatial dimensions
model.add(GlobalAveragePooling2D())

# Add custom dense layers for classification
model.add(Dense(220, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy',precision_metric])

In [30]:
# Define a callback to save the weights when precision is highest
checkpoint = ModelCheckpoint('best_weights.h5',
                             monitor='val_precision_3',
                             save_best_only=True,
                             mode='max',
                             verbose=1)

In [31]:
history=model.fit(train_generator,validation_data=(test_generator),epochs=20,batch_size=len(train_generator),callbacks=[checkpoint])

Epoch 1/20
43/43 [==============================] - ETA: 0s - loss: 0.5934 - accuracy: 0.6972 - precision_3: 0.5036
Epoch 1: val_precision_3 improved from -inf to 0.80000, saving model to best_weights.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


43/43 [==============================] - 90s 2s/step - loss: 0.5934 - accuracy: 0.6972 - precision_3: 0.5036 - val_loss: 0.5502 - val_accuracy: 0.7237 - val_precision_3: 0.8000
Epoch 2/20
43/43 [==============================] - ETA: 0s - loss: 0.4979 - accuracy: 0.7422 - precision_3: 0.5969
Epoch 2: val_precision_3 did not improve from 0.80000
43/43 [==============================] - 86s 2s/step - loss: 0.4979 - accuracy: 0.7422 - precision_3: 0.5969 - val_loss: 0.4848 - val_accuracy: 0.7895 - val_precision_3: 0.7097
Epoch 3/20
43/43 [==============================] - ETA: 0s - loss: 0.4403 - accuracy: 0.7747 - precision_3: 0.6469
Epoch 3: val_precision_3 did not improve from 0.80000
43/43 [==============================] - 89s 2s/step - loss: 0.4403 - accuracy: 0.7747 - precision_3: 0.6469 - val_loss: 0.6319 - val_accuracy: 0.7171 - val_precision_3: 0.7500
Epoch 4/20
43/43 [==============================] - ETA: 0s - loss: 0.4447 - accuracy: 0.7792 - precision_3: 0.6592
Epoch 4: val_

In [33]:
# Load the model with best weights
mdl=load_model("/content/best_weights.h5")

In [34]:
# Evaluate on test_generator
mdl.evaluate(test_generator)

5/5 [==============================] - 10s 1s/step - loss: 0.5655 - accuracy: 0.7105 - precision_3: 1.0000


[0.5655026435852051, 0.7105262875556946, 1.0]

In [57]:
def makeup_detector(img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))  # Adjust target_size as needed
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, 0)  # Add batch dimension
    img_array /= 255.0  # Normalize pixel values to the range [0, 1]
    # Make predictions
    predictions = mdl.predict(img_array)

    # The model has two classes (makeup and no_makeup)
    class_labels = ['no_makeup', 'makeup']
    if predictions[0,0]>0.5:
      print(f"The predicted class for the given image is: {class_labels[1]}")
    else:
      print(f"The predicted class for the given image is: {class_labels[0]}")

In [58]:
# Example usage
img_path="/content/test/no_makeup/no_makeup999.jpeg"
makeup_detector(img_path)

1/1 [==============================] - 0s 58ms/step
The predicted class for the given image is: no_makeup
